**Troubleshooting Scripts - Plan Cache**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Check if _Optimize for Adhoc Workloads_ is enabled**

In [ ]:
SELECT *
FROM sys.configurations WITH (NOLOCK)
WHERE name = 'optimize for ad hoc workloads'
OPTION (MAXDOP 1, RECOMPILE);

**Enable _Optimize for Adhoc Workloads_**

In [ ]:
EXEC sys.sp_configure N'show advanced options', N'1' RECONFIGURE WITH OVERRIDE
GO
EXEC sys.sp_configure N'optimize for ad hoc workloads', N'1' RECONFIGURE WITH OVERRIDE
GO
EXEC sys.sp_configure N'show advanced options', N'0'  RECONFIGURE WITH OVERRIDE
GO

**Ad-hoc Query Plans** 

First query returns total memory consumption of ad-hoc plans. Second query returns 25 largest ad-hoc plans cached

In [ ]:
SELECT 
    CONVERT(DECIMAL(12,3),SUM(1. * p.size_in_bytes)/1024.) AS [Size (KB)]
FROM 
    sys.dm_exec_cached_plans p WITH (NOLOCK)
WHERE 
    p.objtype = 'Adhoc'
OPTION (MAXDOP 1, RECOMPILE);

SELECT TOP 25
    DB_NAME(t.dbid) as [DB]
    ,cp.usecounts
    ,cp.plan_handle
    ,t.[text] 
    ,cp.objtype
    ,cp.size_in_bytes
    ,CONVERT(DECIMAL(12,3),cp.size_in_bytes/1024.) as [Size (KB)]
FROM
    sys.dm_exec_cached_plans cp WITH (NOLOCK)
        CROSS APPLY sys.dm_exec_sql_text(cp.plan_handle) t
WHERE	
    cp.objtype = N'Adhoc'
ORDER BY 
    cp.size_in_bytes DESC 
OPTION (MAXDOP 1, RECOMPILE);

**Single-Used Plans**

First query returns total memory consumption of single-used plans. Second query returns 25 single-used plans cached

In [ ]:
SELECT
   COUNT(*) AS [Single-used plan count]
   ,CONVERT(DECIMAL(10,3)
      ,SUM(CONVERT(BIGINT,cp.size_in_bytes))/1024./1024.) 
          AS [Size (MB)]
FROM
   sys.dm_exec_cached_plans cp WITH (NOLOCK)
WHERE	
   cp.objtype in (N'Adhoc', N'Prepared') AND 
   cp.usecounts = 1
OPTION (MAXDOP 1, RECOMPILE);

SELECT TOP 25
    DB_NAME(t.dbid) as [DB]
    ,cp.usecounts
    ,cp.plan_handle
    ,t.[text] 
    ,cp.objtype
    ,cp.size_in_bytes
    ,CONVERT(DECIMAL(12,3),cp.size_in_bytes/1024.) as [Size (KB)]
FROM
    sys.dm_exec_cached_plans cp WITH (NOLOCK)
        CROSS APPLY sys.dm_exec_sql_text(cp.plan_handle) t
WHERE	
    cp.cacheobjtype = N'Compiled Plan'
    AND cp.objtype in (N'Adhoc', N'Prepared')
    AND cp.usecounts = 1
ORDER BY 
    cp.size_in_bytes DESC 
OPTION (MAXDOP 1, RECOMPILE);

**Remove Plans From Plan Cache - Remove The Single Plan**

Provide plan\_handle as parameter

In [ ]:
DBCC FREEPROCCACHE(0x00) -- Replace with valid plan_handle

**Remove Plans From Plan Cache - Remove All Ad-Hoc Plans**

In [ ]:
DBCC FREESYSTEMCACHE('SQL Plans') WITH MARK_IN_USE_FOR_REMOVAL;

**Get Number of Plan Cache Objects and Their Memory Usage**

In [ ]:
-- Number of cached object and their memory usage grouped by type
SELECT 
    cacheobjtype
    ,objtype 
    ,COUNT(*) AS [Count]
    ,CONVERT(DECIMAL(15,3),SUM(CONVERT(BIGINT,size_in_bytes))/1024./1024.) AS [Size (MB)] 
FROM 
    sys.dm_exec_cached_plans WITH (NOLOCK)
GROUP BY 
    cacheobjtype, objtype
ORDER BY
    [Size (MB)] DESC
OPTION (MAXDOP 1, RECOMPILE);